In [1]:
from bs4 import BeautifulSoup
import os
import cv2

# 导入数据

In [2]:
class_label = ['holothurian', 'echinus', 'scallop', 'starfish', 'waterweeds']
file_path = '../data/train/box/'
image_path = '../data/train/augment/'
all_name = os.listdir(file_path)

# 数据处理

In [3]:
# train_result用于后续yolo训练,k_mean_result用于计算anchor boxes
train_result = []
k_mean_result = []
box_size = {}
label_number = {}
del_number = 0

# 遍历图片
for i in all_name:
    image_name = image_path+i.rstrip('xml')+'jpg'
    img_name_append = False
    soup = BeautifulSoup(open(file_path+i), 'lxml')
    bbx = soup.find_all('object')
    img = cv2.imread(image_name)
    height, width, _ = img.shape
    
    # 对各尺寸数量进行统计
    height, width, _ = img.shape
    if box_size.get((height, width), False):
        box_size[(height, width)] += 1
    else:
        box_size[(height, width)] = 1
    
    # 遍历候选框
    for j in bbx:
        # 同时对范围进行修正，像素起点改为0
        name = str(j.contents[1].string)
        xmin = int(j.xmin.string) - 1
        ymin = int(j.ymin.string) - 1
        xmax = int(j.xmax.string) - 1
        ymax = int(j.ymax.string) - 1
        index = str(class_label.index(name))

        # 防止大小混淆
        if xmin > xmax:
            xmax, xmin = xmin, xmax
        if ymin > ymax:
            ymax, ymin = ymin, ymax
            
        # 判断ground truth是否超出图像范围
        if xmin > width - 1 or xmax < 0:
            continue
        if ymin > height - 1 or ymax < 0:
            continue

        # 处于图像边缘候选框的处理
        if xmin < 0:
            xmin = 0
        if ymin < 0:
            ymin = 0
        if xmax > width - 1:
            xmax = width - 1
        if ymax > height - 1:
            ymax = height - 1
            
        # 判断是否为无效框
        x_distance = xmax - xmin
        y_distance = ymax - ymin
        if x_distance == 0 or y_distance == 0:
            continue
            
        # 对于过小的图片进行删除
        square = (xmax - xmin) * (ymax - ymin)
        if square < 120:
            del_number += 1
            continue
        
        # 防止有图片没有任何候选框
        if img_name_append == False:
            train_result.append(image_name)
            img_name_append = True

        # 判断是否增加kmean缩放候选框
        scale = min(448/width, 448/height)
        square = ((xmax - xmin)*scale) * ((ymax - ymin)*scale)
        if square < 120:
            kmean_scale = False
        else:
            kmean_scale = True
        
        xmin, xmax, ymin, ymax = str(xmin), str(xmax), str(ymin), str(ymax)
        train_result.append(' ')
        for x in [xmin, ymin, xmax, ymax]:
            train_result.append(x)
            train_result.append(',')
            if x is ymax:
                train_result.append(index)
        
        # kmean添加缩放候选框
        if kmean_scale == True:
            for x in [xmin, ymin, xmax, ymax]:
                k_mean_result.append(int(round(int(x)*scale)))
                if x is not ymax:
                    k_mean_result.append(',')
                else:
                    k_mean_result.append('\n')
        
        # 对各label数量统计
        if label_number.get(name, False):
            label_number[name] += 1
        else:
            label_number[name] = 1
    if img_name_append == True:
        train_result.append('\n')

# 查看统计结果

In [4]:
del_number

10

In [5]:
label_number

{'echinus': 22098,
 'scallop': 6694,
 'starfish': 6827,
 'holothurian': 5503,
 'waterweeds': 82}

In [6]:
box_size

{(405, 720): 3066,
 (2160, 3840): 1644,
 (1080, 1920): 595,
 (576, 704): 38,
 (480, 586): 44}

# 保存

In [7]:
with open('../data/train_data.txt', 'w') as f:
    f.writelines(train_result)
with open('../data/k_mean_data.txt', 'w') as f:
    f.writelines(k_mean_result)